In [ ]:
# Perform Synthesizer Parameter Search

In [1]:
import IPython.display as ipd

from ohc.search.search import EvolutionarySearch
from ohc.search.search import FindMySound
from ohc.vst import VSTHost
from ohc.fitness.fitness import FitnessFunction
from ohc.fitness.clap import CLAPSimilarity

%load_ext autoreload
%autoreload 2

/Users/svanka/miniforge3/envs/ohc/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
2024-02-25 15:01:05,823	INFO util.py:154 -- Missing packages: ['ipywidgets']. Run `pip install -U ipywidgets`, then restart the notebook server for rich notebook output.


In [19]:
#check if gpu is available
import torch
print("GPU Available: ", torch.cuda.is_available()) 

GPU Available:  False


Exception ignored in: <function Image.__del__ at 0x16b695f30>
Traceback (most recent call last):
  File "/Users/svanka/miniforge3/envs/ohc/lib/python3.10/tkinter/__init__.py", line 4056, in __del__
    self.tk.call('image', 'delete', self.name)
RuntimeError: main thread is not in main loop
Exception ignored in: <function Variable.__del__ at 0x16b6267a0>
Traceback (most recent call last):
  File "/Users/svanka/miniforge3/envs/ohc/lib/python3.10/tkinter/__init__.py", line 388, in __del__
    if self._tk.getboolean(self._tk.call("info", "exists", self._name)):
RuntimeError: main thread is not in main loop
Exception ignored in: <function Variable.__del__ at 0x16b6267a0>
Traceback (most recent call last):
  File "/Users/svanka/miniforge3/envs/ohc/lib/python3.10/tkinter/__init__.py", line 388, in __del__
    if self._tk.getboolean(self._tk.call("info", "exists", self._name)):
RuntimeError: main thread is not in main loop
Exception ignored in: <function Variable.__del__ at 0x16b6267a0>
Traceb

In [5]:
PLUGIN_PATH = "/Library/Audio/Plug-Ins/VST3/Dexed.vst3"
SAMPLE_RATE = 48000
BLOCK_SIZE = 512

In [6]:
# Create a VST
vst = VSTHost(PLUGIN_PATH, "fixed", SAMPLE_RATE, BLOCK_SIZE)

# Select parameters to search over
params = vst.list_params(filter_midi_cc=True)
active_params = [p for p in params if p != "Bypass"]
vst.set_active_params(active_params)

error: attempt to map invalid URI `/Library/Audio/Plug-Ins/VST3/Dexed.vst3'
2024-02-25 15:01:37,253	INFO worker.py:1724 -- Started a local Ray instance.


In [7]:
# Initialize the CLAP embedding
clap = CLAPSimilarity(sample_rate=SAMPLE_RATE, device = 'cpu')

Initializing CLAP model: laion/larger_clap_general
Initializing CLAP tokenizer: laion/larger_clap_general
Initializing CLAP feature extractor: laion/larger_clap_general


In [13]:
# Create a fitness function
text_target = ["snoring in sleep"]
fitness = FitnessFunction(vst, clap, text_target=text_target, clap_batch_size=128)

In [14]:
problem = FindMySound(
    "max",
    num_params=len(active_params),
    min_value=0.0,
    max_value=1.0,
    fitness_function=fitness,
)

[2024-02-25 15:04:08] INFO     <85875> evotorch.core: Instance of `FindMySound` (id:14276837680) -- The `dtype` for the problem's decision variables is set as torch.float32
[2024-02-25 15:04:08] INFO     <85875> evotorch.core: Instance of `FindMySound` (id:14276837680) -- `eval_dtype` (the dtype of the fitnesses and evaluation data) is set as torch.float32
[2024-02-25 15:04:08] INFO     <85875> evotorch.core: Instance of `FindMySound` (id:14276837680) -- The `device` of the problem is set as cpu
[2024-02-25 15:04:08] INFO     <85875> evotorch.core: Instance of `FindMySound` (id:14276837680) -- The number of actors that will be allocated for parallelized evaluation is 0


In [24]:
search = EvolutionarySearch(problem, population_size=100, max_generations=20, logger="custom")

In [25]:
search.run()

Processing batch of 50 audio clips
<LazyStatusDict
    center = tensor([0.7205, 0.2946, 0.6644, 0.3870, 0.9444, 0.9724, 0.6284, 0.2807, 0.8235,
        0.6168, 0.4286, 0.3527, 0.4964, 0.8342, 0.6311, 0.0152, 0.0069, 0.5579,
        0.7324, 0.0020, 0.1746, 0.8542, 0.4728, 0.9522, 0.4984, 0.5305, 0.2199,
        0.1576, 0.1965, 0.6262, 0.1263, 0.6768, 0.5924, 0.5653, 0.1533, 0.4876,
        0.0258, 0.9181, 0.4751, 0.9471, 0.5881, 0.3891, 0.8688, 0.2571, 0.5115,
        0.4528, 0.6357, 0.9464, 0.7535, 0.0117, 0.5797, 0.3985, 0.1419, 0.5483,
        0.7941, 0.7943, 0.0261, 0.8103, 0.3644, 0.6392, 0.1315, 0.0333, 0.7248,
        0.2747, 0.1595, 0.5504, 0.3902, 0.1045, 0.9107, 0.1292, 0.2038, 0.8424,
        0.3391, 0.3835, 0.0180, 0.4891, 0.3968, 0.9295, 0.1532, 0.9905, 0.4043,
        0.9685, 0.8977, 0.2165, 0.1324, 0.6135, 0.2690, 0.1179, 0.9550, 0.2851,
        0.0235, 0.0187, 0.4465, 0.3139, 0.1763, 0.8812, 0.3833, 0.7576, 0.8280,
        0.9807, 0.1381, 0.0411, 0.9279, 0.7275, 0.5785, 

KeyboardInterrupt: 

In [ ]:
best_discovered_solution = search._searcher.status["best"]

In [ ]:
audio, _ = vst.render_now(best_discovered_solution.values.unsqueeze(0), 48, 1.0, 1.0)[0]

In [ ]:
ipd.Audio(audio, rate=SAMPLE_RATE)